In [1]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import scipy.io
from scipy.interpolate import griddata

!pip install pyDOE
from pyDOE import lhs

from collections import OrderedDict
from tqdm import tqdm

sns.set_style("white")

C:\Users\ameyv\anaconda3\envs\amey-pinn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Multi-layer Perceptron (Deep Neural Network)
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        
        self.depth = len(layers) - 1
        self.activation = torch.nn.Tanh
        
        layer_list = list()
        for i in range(self.depth - 1): 
            layer_list.append(('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1])))
            layer_list.append(('activation_%d' % i, self.activation()))
            
        layer_list.append(('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1])))
        layerDict = OrderedDict(layer_list)
        
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [3]:
# Physics Informed Neural Network
class PINN():
    def __init__(self, X_s, s, X_f, layers, lb, ub, nu):
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

        # boundary conditions
        self.lb = torch.tensor(lb).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)
        
        # data
        self.x_s = torch.tensor(X_s[:, 0:1], requires_grad=True).float().to(device)
        self.y_s = torch.tensor(X_s[:, 1:2], requires_grad=True).float().to(device)
        self.x_f = torch.tensor(X_f[:, 0:1], requires_grad=True).float().to(device)
        self.y_f = torch.tensor(X_f[:, 1:2], requires_grad=True).float().to(device)
        self.s = torch.tensor(s).float().to(device)
        
        # deep neural networks
        self.layers = layers
        self.nu = nu
        self.dnn = DNN(layers).to(device)
        
        # optimizers: using the same settings
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(), 
            lr=1.0, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=50,
            tolerance_grad=1e-5, 
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe"      
        )
        self.iter = 0

    def net_s(self, x, y):  
        s = self.dnn(torch.cat([x, y], dim=1))
        return s
    
    def net_f(self, x, y):
        s = self.net_s(x, y)
        u = s[:,0]
        v = s[:,1]
        p = s[:,2]

        du_dx = torch.autograd.grad(inputs=x, outputs=u, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
        dv_dx = torch.autograd.grad(inputs=x, outputs=v, grad_outputs=torch.ones_like(v), retain_graph=True, create_graph=True)[0]
        dp_dx = torch.autograd.grad(inputs=x, outputs=p, grad_outputs=torch.ones_like(p), retain_graph=True, create_graph=True)[0]
        
        du_dy = torch.autograd.grad(inputs=y, outputs=u, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
        dv_dy = torch.autograd.grad(inputs=y, outputs=v, grad_outputs=torch.ones_like(v), retain_graph=True, create_graph=True)[0]
        dp_dy = torch.autograd.grad(inputs=y, outputs=p, grad_outputs=torch.ones_like(p), retain_graph=True, create_graph=True)[0]

        du_dxx = torch.autograd.grad(inputs=x, outputs=du_dx, grad_outputs=torch.ones_like(du_dx), retain_graph=True, create_graph=True)[0]
        du_dyy = torch.autograd.grad(inputs=y, outputs=du_dy, grad_outputs=torch.ones_like(du_dy), retain_graph=True, create_graph=True)[0]
        dv_dxx = torch.autograd.grad(inputs=x, outputs=dv_dx, grad_outputs=torch.ones_like(dv_dx), retain_graph=True, create_graph=True)[0]
        dv_dyy = torch.autograd.grad(inputs=y, outputs=dv_dy, grad_outputs=torch.ones_like(dv_dy), retain_graph=True, create_graph=True)[0]
        
        f1 = u.squeeze() * du_dx + v.squeeze() * du_dy + dp_dx - self.nu * (du_dxx + du_dyy)
        f2 = u.squeeze() * dv_dx + v.squeeze() * dv_dy + dp_dy - self.nu * (dv_dxx + dv_dyy)
        f = f1 + f2
        return f
    
    def loss_func(self):
        self.optimizer.zero_grad()
        
        s_pred = self.net_s(self.x_s, self.y_s)
        f_pred = self.net_f(self.x_f, self.y_f)
#         print("s:", type(self.s))
#         print("f_pred:", type(f_pred))
#         print("s_pred:", type(s_pred))
        loss_s = torch.mean((self.s - s_pred) ** 2)
        loss_f = torch.mean(f_pred ** 2)
        
        loss = loss_s + loss_f
        
        loss.backward()
        self.iter += 1
        if self.iter % 100 == 0:
            print('Iter %d, Loss: %.5e, Loss_u: %.5e, Loss_f: %.5e' % (self.iter, loss.item(), loss_s.item(), loss_f.item()))
        return loss
    
    def train(self):
        self.dnn.train()
        self.optimizer.step(self.loss_func)
     
    def predict(self, X):
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        y = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)

        self.dnn.eval()
        s = self.net_s(x, y)
        f = self.net_f(x, y)
        s = s.detach().cpu().numpy()
        f = f.detach().cpu().numpy()
        return s, f

In [4]:
N_u = 500
N_f = 5000

Re = 20
nu = 1 / Re
lamb = 1 / (2 * nu) - np.sqrt(1 / (4 * nu ** 2) + 4 * np.pi ** 2)

layers = [2, 30, 30, 30, 30, 30, 30, 3]

h = 0.005
k = 0.005
x = torch.arange(0, 1, h)
y = torch.arange(0, 1, k)

print("x:", x.shape)
print("y:", y.shape)

X, Y = np.meshgrid(x,y)
X = torch.from_numpy(X)
Y = torch.from_numpy(Y)

print("X:", X.shape)
print("Y:", Y.shape)

Exact = torch.zeros(X.shape[0],X.shape[1],3)
Exact_U = 1-(torch.exp(-lamb*X))*(torch.cos(2*math.pi*Y))
Exact_V = (lamb/(2*math.pi))*(torch.exp(lamb*X))*(torch.sin(2*math.pi*X))
Exact_P = (1/2)*(1-(torch.exp(2*lamb*X)))
Exact[:,:,0] = Exact_U
Exact[:,:,1] = Exact_V
Exact[:,:,2] = Exact_P

print("Exact:",Exact.shape)

X_test = np.hstack((X.flatten()[:,None], Y.flatten()[:,None]))
s_test = torch.reshape(Exact, (-1, 3))             

print("X_test:", X_test.shape)
print("s_test:", s_test.shape)

# Domain bounds
lb = X_test.min(0)
ub = X_test.max(0) 


idx = np.random.choice(1000, N_u, replace=False)
X_s_train = X_test[idx, :]
s_train   = s_test[idx, :]

print("X_s_train:", X_s_train.shape)
print("s_train:", s_train.shape)

X_f_train = lb + (ub-lb)*lhs(2, N_f)
X_f_train = np.vstack((X_f_train, X_s_train))

print("X_f_train:", X_f_train.shape)

x: torch.Size([200])
y: torch.Size([200])
X: torch.Size([200, 200])
Y: torch.Size([200, 200])
Exact: torch.Size([200, 200, 3])
X_test: (40000, 2)
s_test: torch.Size([40000, 3])
X_s_train: (500, 2)
s_train: torch.Size([500, 3])
X_f_train: (5500, 2)


In [5]:
model = PINN(X_s_train, s_train, X_f_train, layers, lb, ub, nu)
model.train()

C:\Users\ameyv\AppData\Local\Temp\ipykernel_5984\1660453502.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.s = torch.tensor(s).float().to(device)


Iter 100, Loss: 1.35593e-01, Loss_u: 1.10042e-01, Loss_f: 2.55512e-02
Iter 200, Loss: 9.08227e-02, Loss_u: 8.46753e-02, Loss_f: 6.14740e-03
Iter 300, Loss: 8.73745e-02, Loss_u: 8.52080e-02, Loss_f: 2.16652e-03
Iter 400, Loss: 8.50999e-02, Loss_u: 8.20708e-02, Loss_f: 3.02906e-03
Iter 500, Loss: 8.42041e-02, Loss_u: 8.19362e-02, Loss_f: 2.26789e-03
Iter 600, Loss: 8.29125e-02, Loss_u: 8.04520e-02, Loss_f: 2.46049e-03
Iter 700, Loss: 8.17817e-02, Loss_u: 7.80997e-02, Loss_f: 3.68208e-03
Iter 800, Loss: 8.04997e-02, Loss_u: 7.48100e-02, Loss_f: 5.68974e-03
Iter 900, Loss: 7.87158e-02, Loss_u: 6.98795e-02, Loss_f: 8.83627e-03
Iter 1000, Loss: 7.66567e-02, Loss_u: 6.79481e-02, Loss_f: 8.70854e-03
Iter 1100, Loss: 7.24238e-02, Loss_u: 6.64732e-02, Loss_f: 5.95060e-03
Iter 1200, Loss: 6.72201e-02, Loss_u: 6.27092e-02, Loss_f: 4.51085e-03
Iter 1300, Loss: 6.46569e-02, Loss_u: 6.07949e-02, Loss_f: 3.86198e-03
Iter 1400, Loss: 6.08159e-02, Loss_u: 5.58646e-02, Loss_f: 4.95133e-03
Iter 1500, Loss